In [1]:

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import pandas as pd
import argparse

for device in tf.config.list_physical_devices():
    print(": {}".format(device.name))


: /physical_device:CPU:0


In [ ]:

#2. Configuration and Data Path Setup
# Configuration parameters
IMAGE_SIZE = (224, 224)  # Reduced size for faster training
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.0001
N_CLASSES = 2

def get_dataset_path():
    """Get dataset path from command line argument or use default"""
    try:
        parser = argparse.ArgumentParser(description='Wildfire Detection Model')
        parser.add_argument('--dataset_path', type=str, default=r'C:\Users\Admin\Desktop\home files\PATHWAYS\Wildfire Prediction Model\archive (3)',
                          help='Path to the wildfire dataset')
        args, _ = parser.parse_known_args()
        return args.dataset_path
    except:
        # Default path if no argument is provided
        return r'C:\Users\Admin\Desktop\home files\PATHWAYS\Wildfire Prediction Model\archive (3)'


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 3-4: truncated \UXXXXXXXX escape (1724049977.py, line 13)

In [ ]:
dataset_path = get_dataset_path()

train_path = os.path.join(dataset_path, 'train')
valid_path = os.path.join(dataset_path, 'validation')  
test_path = os.path.join(dataset_path, 'test')

print(f"Dataset path: {dataset_path}")
print(f"Train path: {train_path}")
print(f"Validation path: {valid_path}")
print(f"Test path: {test_path}")

for path in [train_path, valid_path, test_path]:
    if os.path.exists(path):
        print(f"✓ {path} exists")
        classes = os.listdir(path)
        print(f"  Classes found: {classes}")
        for class_name in classes:
            class_path = os.path.join(path, class_name)
            if os.path.isdir(class_path):
                num_images = len([f for f in os.listdir(class_path) if f.endswith(('.jpg', '.jpeg', '.png'))])
                print(f"  {class_name}: {num_images} images")
    else:
        print(f"✗ {path} does not exist")


In [ ]:

CLASS_NAMES = ['nowildfire', 'wildfire']
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest',
    dtype='float32'
)


In [ ]:

valid_test_datagen = ImageDataGenerator(rescale=1./255, dtype='float32')
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=CLASS_NAMES,
    shuffle=True
)

valid_generator = valid_test_datagen.flow_from_directory(
    valid_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=CLASS_NAMES,
    shuffle=False
)

test_generator = valid_test_datagen.flow_from_directory(
    test_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=CLASS_NAMES,
    shuffle=False
)

print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {valid_generator.samples}")
print(f"Test samples: {test_generator.samples}")


In [ ]:

weight_decay = 1e-4  

model = Sequential([
    # First Convolutional Block
    Conv2D(32, (3, 3), activation='relu', padding='same', 
           input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
           kernel_regularizer=regularizers.l2(weight_decay)),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu', padding='same',
           kernel_regularizer=regularizers.l2(weight_decay)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    # Second Convolutional Block
    Conv2D(64, (3, 3), activation='relu', padding='same',
           kernel_regularizer=regularizers.l2(weight_decay)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same',
           kernel_regularizer=regularizers.l2(weight_decay)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    # Third Convolutional Block
    Conv2D(128, (3, 3), activation='relu', padding='same',
           kernel_regularizer=regularizers.l2(weight_decay)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(N_CLASSES, activation='softmax')
])


In [ ]:

model.summary()


In [ ]:

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

checkpointer = ModelCheckpoint('wildfire_model_best.h5', verbose=1, 
                              save_best_only=True, monitor='val_accuracy')
early_stopping = EarlyStopping(monitor='val_loss', patience=15, 
                              restore_best_weights=True, verbose=1)
optimizer = optimizers.Adam(learning_rate=LEARNING_RATE)

model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy', 'AUC', 'Precision', 'Recall']
)


In [ ]:

from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))
print(f"Class weights: {class_weights}")


In [ ]:

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    verbose=1,
    validation_data=valid_generator,
    callbacks=[checkpointer, early_stopping],
    class_weight=class_weights
)


In [ ]:

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:

test_results = model.evaluate(test_generator, verbose=1)
print(f"Test Loss: {test_results[0]:.4f}")
print(f"Test Accuracy: {test_results[1]:.4f}")
print(f"Test AUC: {test_results[2]:.4f}")


In [ ]:

model.save('wildfire_detection_model_final.h5')
print("Model saved as 'wildfire_detection_model_final.h5'")


In [ ]:


def predict_wildfire(image_path, model):
    """Predict if an image contains wildfire"""

    img = load_img(image_path, target_size=IMAGE_SIZE)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    
    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction)
    confidence = prediction[0][class_idx]

    class_name = CLASS_NAMES[class_idx]
    return class_name, confidence


In [ ]:

# Example usage:
result, confidence = predict_wildfire('path_to_image.jpg', model)
print(f"Prediction: {result} (Confidence: {confidence:.2f})")